# Time series

A time series is a series with dates as index.

   1. [Creation & use](#creation)
   2. [Data manipulation](#manipulation)
   3. [Examples](#examples)

Note: see what is a date/time in Python if you haven't: [Introduction to DateTime](../lesson2 Deeper in Python/21 datetime.ipynb)

Ref: http://pandas.pydata.org/pandas-docs/stable/timeseries.html

# Tableaux chronologiques

On peut choisir d'indexer son tableau par un index chronologique. Dans ce cas certaines opérations
liées au temps deviennent possibles.

Note: regardez ce qu'est une date/heure dans Python si vous ne l'avez pas encore fait : [Introduction à DateTime](../lesson2 Deeper in Python/21 datetime.ipynb)

<a id='creation'></a>
## Creation and uses

Dates are ordered and can be given in a list of dates, as any index, or  with the methods `pd.date_range(start, end, periode, frequence)` where 

* you should choose between end and period, period being the number of iteration
* frequence is define by these acronymes:

```
B    business day frequency
C    custom business day frequency (experimental)
D    calendar day frequency
W    weekly frequency
M    month end frequency
BM   business month end frequency
CBM  custom business month end frequency
MS   month start frequency
BMS  business month start frequency
CBMS custom business month start frequency
Q    quarter end frequency
BQ   business quarter end frequency
QS   quarter start frequency
BQS  business quarter start frequency
A    year end frequency
BA   business year end frequency
AS   year start frequency
BAS  business year start frequency
BH   business hour frequency
H    hourly frequency
T, min minutely frequency
S      secondly frequency
L, ms  milliseconds
U, us  microseconds
N      nanoseconds
```

see http://pandas.pydata.org/pandas-docs/stable/timeseries.html

<a id='creation'></a>
## Création et utilisation

Les dates sont ordonnées et peuvent être données dans une liste de dates, sous forme d’index, ou avec la méthode

`pd.date_range (début, fin, période, fréquence)`

où vous devez choisir entre fin et période, la période étant le nombre d'itérations

La fréquence est définie par ces codes (attention, ils sont différents de ceux de Numpy datetime) :

```
B        jours ouvrables
C        jours ouvrables personnalisée (à titre expérimental)
J        jour
W        hebdomadaire
M        mensuel
BM       mois des affaires
CBM      mois d'activité personnalisée
MS       début de mois
BMS      début de mois d'activité
CBMS     début de mois d'activité personnalisée
Q        fin de trimestre
BQ       trimestre d’affaires
QS       début de trimestre
BQS      début du trimestre d'activité
A        fin d'année
BA       fin d'année d'exercice
AS       début d'année 
BAS      année de début d'exercice
BH       heure professionel
H        heure
T, min   minute
S        seconde
L, ms    millisecondes
U, us    microsecondes
N        nanosecondes
```

In [1]:
import numpy as np
import pandas as pd
np.random.seed(1)

In [2]:
dates = pd.date_range('2016-08-28', '2016-09-06', freq='B') # begin, end, only business days
dates

DatetimeIndex(['2016-08-29', '2016-08-30', '2016-08-31', '2016-09-01',
               '2016-09-02', '2016-09-05', '2016-09-06'],
              dtype='datetime64[ns]', freq='B')

Avec cet index on peut créer un tableau chronologique :

It is easy to extract parts of a TimeSeries:

In [3]:
tdf1 = pd.DataFrame({'temperature': 20 + np.random.randint(0,5,7),
                     'pression'   : 1 + np.random.random(7)/10 },
                    index=dates)
tdf1

,temperature,pression
2016-08-29,23,1.039658
2016-08-30,24,1.038791
2016-08-31,20,1.066975
2016-09-01,21,1.093554
2016-09-02,23,1.084631
2016-09-05,20,1.031327
2016-09-06,20,1.052455


Comme pour les tableaux usuels on peut selectionner les parties qui nous intéresse avec `loc` et les filtres
mais il est également possible de contraindre les dates :

In [4]:
tdf1.loc['2016-08']  # just August

,temperature,pression
2016-08-29,23,1.039658
2016-08-30,24,1.038791
2016-08-31,20,1.066975


In [5]:
tdf1.loc['2016-09-03':]  # after that date even if the date is not in the index

,temperature,pression
2016-09-05,20,1.031327
2016-09-06,20,1.052455


<a id='manipulation'></a>
## Manipulation

###  Boucher les trous

Soit deux sources d'information incomplète, utilisons les méthodes que l'on a déjà vu pour boucher les trous.

In [6]:
tdf2 = tdf1.copy()
tdf1.drop(tdf1.index[[0,1,3]], inplace=True)   # we remove some data
tdf2.drop(tdf2.index[[5,6]], inplace=True)         # more data removed
tdf2.drop(columns='pression', inplace=True)
display(tdf1, tdf2)

,temperature,pression
2016-08-31,20,1.066975
2016-09-02,23,1.084631
2016-09-05,20,1.031327
2016-09-06,20,1.052455


,temperature
2016-08-29,23
2016-08-30,24
2016-08-31,20
2016-09-01,21
2016-09-02,23


In [7]:
tdf1.merge(tdf2, on='temperature', how='outer', right_index=True, left_index=True)  # merge on indexes + col temperature

,temperature,pression
2016-08-29,23,NaN
2016-08-30,24,NaN
2016-08-31,20,1.066975
2016-09-01,21,NaN
2016-09-02,23,1.084631
2016-09-05,20,1.031327
2016-09-06,20,1.052455


Si les deux sources de données ne sont pas d'accord sur une valeur, que se passe-t-il ?

In [8]:
tdf1.loc['2016-08-31','temperature'] = 19
tdf1 = tdf1.merge(tdf2, on='temperature', how='outer', right_index=True, left_index=True)
tdf1

,temperature,pression
2016-08-29,23,NaN
2016-08-30,24,NaN
2016-08-31,19,1.066975
2016-09-01,21,NaN
2016-09-02,23,1.084631
2016-09-05,20,1.031327
2016-09-06,20,1.052455


On constate que la nouvelle donnée s'est imposée car :

* on fait la fusion sur les index qui eux sont cohérents
* la fusion est guidée par `tdf1` qui appelle la méthode `merge`


### Interpolation

Comme avec Numpy, l'interpolation peut être faite en prenant en compte les dates et donc l'écart entre 2 dates successives.

In [9]:
tdf1.interpolate(method='time')

,temperature,pression
2016-08-29,23,NaN
2016-08-30,24,NaN
2016-08-31,19,1.066975
2016-09-01,21,1.075803
2016-09-02,23,1.084631
2016-09-05,20,1.031327
2016-09-06,20,1.052455


Il est également possible de changer l'index et de demander de recalculer les valeurs sur le nouvel index.

In [10]:
tdf1.resample('30H').interpolate(method='time')

,temperature,pression
2016-08-29 00:00:00,23.0,NaN
2016-08-30 06:00:00,23.0,NaN
2016-08-31 12:00:00,23.0,NaN
2016-09-01 18:00:00,23.0,NaN
2016-09-03 00:00:00,23.0,NaN
2016-09-04 06:00:00,23.0,NaN
2016-09-05 12:00:00,23.0,NaN


Ce qui n'est pas le résultat attendu (ni pour la température, ni pour la pression). Pandas a un bug ([référencé](https://github.com/pandas-dev/pandas/issues?utf8=%E2%9C%93&q=is%3Aissue+is%3Aopen+label%3ATimeseries++label%3ABug) avec d'autres)
aussi en attendant sa correction voici une longue méthode pour avoir le bon résultat :

In [11]:
df1 = pd.DataFrame(tdf1)
df2 = pd.DataFrame(tdf1.asfreq('30H'))
tmp = pd.concat([df1, df2]).sort_index().interpolate(method='time').drop_duplicates()
display(tmp)
pd.merge(tmp, df2, how='inner', on=list(tmp.columns), left_index=True, right_index=True)

,temperature,pression
2016-08-29 00:00:00,23.00,NaN
2016-08-30 00:00:00,24.00,NaN
2016-08-30 06:00:00,22.75,NaN
2016-08-31 00:00:00,19.00,1.066975
2016-08-31 12:00:00,20.00,1.071389
2016-09-01 00:00:00,21.00,1.075803
2016-09-01 18:00:00,22.50,1.082424
2016-09-02 00:00:00,23.00,1.084631
2016-09-03 00:00:00,22.00,1.066863
2016-09-04 06:00:00,20.75,1.044653


,temperature,pression
2016-08-29 00:00:00,23.00,NaN
2016-08-30 06:00:00,22.75,NaN
2016-08-31 12:00:00,20.00,1.071389
2016-09-01 18:00:00,22.50,1.082424
2016-09-03 00:00:00,22.00,1.066863
2016-09-04 06:00:00,20.75,1.044653
2016-09-05 12:00:00,20.00,1.041891


### Grouper les données

En utilisant l'index temporel il est possible de grouper les données par périodes. Pour cela on utiliser [`Grouper`](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.Grouper.html) qui
génère les groupes de dates correspondant à la période demandée :

In [12]:
tdf1.groupby(pd.Grouper(freq='W')).mean()

,temperature,pression
2016-09-04,22,1.075803
2016-09-11,20,1.041891


## Plus

Pour plus d'information sur les tableaux chronologiques on regardera la page sur les séries chronologiques :
http://pandas.pydata.org/pandas-docs/stable/timeseries.html

{{ PreviousNext("pd06 -- Merging 2 dataframes.ipynb","pd08 -- Tools.ipynb")}}